## Import

In [73]:
from tqdm import tqdm
from datetime import timedelta, datetime, date

import streamlit as st
import pandas as pd
import numpy as np
import OpenDartReader
import warnings
import dart_fss
import time
import re, os

warnings.filterwarnings('ignore')

## 초기 세팅
api_key = 'f54ffa528f132c33a392456a76438073353fa2b5'
dart = OpenDartReader(api_key)
dart_fss.set_api_key(api_key=api_key)

## 현재 기준 상장 기업 리스트 추출
corp_dict = dart_fss.api.filings.get_corp_code()
corp_df = pd.DataFrame(corp_dict)
corp_df = corp_df.loc[corp_df.stock_code.notnull()]
corp_df.index = [x for x in range(corp_df.shape[0])]

## 분기 > 날짜 변환 관련 dictionary
market_dict = {"Y": "코스피","K": "코스닥", "N": "코넥스", "E": "기타"}
q_dict = {"1분기": ['{}-01-01', '{}-03-31'],
          "2분기": ['{}-04-01', '{}-06-30'],
          "3분기": ['{}-07-01', '{}-09-30'],
          "4분기": ['{}-10-01', '{}-12-31']}

change_dict = {"1분기보고서": 11013, "반기보고서": 11012, "3분기보고서": 11014, "사업보고서":11011}

## 폴더 생성
data_path = './datasets/'
if not os.path.isdir(data_path):
    os.mkdir(data_path)
    
## 보고서별 리스트 추출
loop_list = ['{}분기'.format(x) for x in range(1,5)]

## RPA 가정, 현재 날짜 기준 수집해야하는 보고서 list
today = date.today()
dt_q1 = datetime.strptime('{}-03-31'.format(today.year), '%Y-%m-%d') + timedelta(days = 50)
dt_q2 = datetime.strptime('{}-06-30'.format(today.year), '%Y-%m-%d') + timedelta(days = 50)
dt_q3 = datetime.strptime('{}-09-30'.format(today.year), '%Y-%m-%d') + timedelta(days = 50)
dt_q4 = datetime.strptime('{}-12-31'.format(today.year), '%Y-%m-%d') + timedelta(days = 95)

year = today.year
loop_cnt = 1

if today <= datetime.date(dt_q1):
    r_list = ['사업보고서', '1분기보고서']
    y_list = [year-1, year]
    loop_cnt = 1
elif (today > datetime.date(dt_q1)) and (today <= datetime.date(dt_q2)):
    r_list = ['1분기보고서', '반기보고서']
    y_list = [year, year]
elif (today > datetime.date(dt_q2)) and (today <= datetime.date(dt_q3)):
    r_list = ['반기보고서', '3분기보고서']
    y_list = [year, year]
elif (today > datetime.date(dt_q3)) and (today <= datetime.date(dt_q4)):
    r_list = ['반기보고서', '사업보고서']
    y_list = [year, year]

for year in y_list[loop_cnt:]:
    for r_code in r_list:
        for idx, q in enumerate(loop_list):
            start_dt, end_dt = [x.format(year) for x in q_dict[q]]

            if r_code == "사업보고서":
                temp_df = dart.list(start=start_dt, end=end_dt, kind_detail = 'A001') # 사업
            elif r_code == "반기보고서":
                temp_df = dart.list(start=start_dt, end=end_dt, kind_detail = 'A002') # 반기
            else:
                temp_df = dart.list(start=start_dt, end=end_dt, kind_detail = 'A003') # 분기

            if idx == 0:
                info_df = temp_df
            else:
                info_df = pd.concat([info_df, temp_df])

        ## 상장 기업들 추출
        corp_list = info_df.loc[info_df.corp_cls.isin(['Y', 'K', 'N'])].corp_name.unique()

        ## 전체 기업 대상 loop를 돌면서 보고서 수집
        cnt = 0
        p_cnt = 0

        for corp in tqdm(corp_list):
            try:
                temp_df = dart.report(corp, '타법인출자', year, change_dict[r_code])
            except ValueError:
                if cnt != 0:
                    cnt += 1
                    p_cnt += 1
                continue
            p_cnt += 1

            if temp_df.shape[0] == 0:
                continue

            elif cnt == 0 and temp_df.shape[0] != 1:
                output = temp_df
                cnt += 1

            else:
                output = pd.concat([output, temp_df])

        ## 단순투자만 추출
        output.invstmnt_purps = [x.replace(" ", "").replace("\n", "") for x in output.invstmnt_purps]
        final_df = output.loc[[True if "단순" == x or "단순투자" in x else False for x in output.invstmnt_purps], :]

        ## 현업 양식에 변환
        select_cols = ['corp_cls', 'corp_name', 'inv_prm', 'frst_acqs_de', 'invstmnt_purps', 'frst_acqs_amount', 'trmend_blce_qy', 'trmend_blce_qota_rt', 'trmend_blce_acntbk_amount']
        change_cols = ['법인구분', '회사명', '법인명', '최초취득일자', '출자목적', '최초취득금액', '기말잔액수량', '기말잔액지분율', '기말잔액장부가액']

        save_df = final_df.loc[:, select_cols]
        save_df.columns = change_cols
        save_df['법인구분'] = save_df['법인구분'].map(market_dict)

        clean_txt = [re.sub(r'\([^)]*\)', '', x) for x in save_df['법인명']]
        clean_txt = [re.sub("㈜", "", x) for x in clean_txt]
        clean_txt = [re.sub("\s", "", x) for x in clean_txt]
        save_df['법인명'] = clean_txt
        check = save_df.loc[save_df['법인명'].isin(list(corp_df.corp_name.unique()))]

        ## 파일 저장
        check.to_excel(data_path + "ECM2부-타법인출자현황-단순투자-{}-{}.xlsx".format(year, r_code), index = False, encoding = "CP949")

{"status":"013","message":"조회된 데이타가 없습니다."}


  0%|                                                                                 | 2/2436 [00:00<05:07,  7.92it/s]

{'status': '013', 'message': '조회된 데이타가 없습니다.'}


  0%|                                                                                 | 3/2436 [00:00<04:56,  8.21it/s]

{'status': '013', 'message': '조회된 데이타가 없습니다.'}


...

 94%|█████████████████████████████████████████████████████████████████████████████     | 47/50 [00:05<00:00,  8.09it/s]

{'status': '013', 'message': '조회된 데이타가 없습니다.'}


 96%|██████████████████████████████████████████████████████████████████████████████▋   | 48/50 [00:05<00:00,  7.77it/s]

{'status': '013', 'message': '조회된 데이타가 없습니다.'}


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:05<00:00,  8.62it/s]
